# URSS Project - Integer Solutions on Cubic Surfaces
## By William Fuller


## 0. Introduction

 
My URSS project is concerned with a branch of Number Theory namely exploring integer solutions on Cubic Surfaces for which I have used computational methods to do so. I have predominantely been using the free to use Computer Algebra System [SageMath](https://www.sagemath.org/), its wide range of applications can be seen in its [reference manual](https://doc.sagemath.org/html/en/reference/index.html).  

I would like to thank my supervisor for this project, Simon Rydin Myerson, for all the help and guidance along the way. 

## 1. Background

In the context of this project, cubic surfaces are a specific type of [diophantine equation](https://en.wikipedia.org/wiki/Diophantine_equation); they are characterised by a degree three polynomial equation with integer coefficients in three variables and we're interested in the integer points on this surface. 
More precisely say we have a degree-3 polynomial $f \in \mathbb{Z}[x,y,z]$ and the equation $f(x, y, z) = k$, $k \in \mathbb{N}$, then the cubic surface which is characterised by this equation is given by, 
$$ 
V_{(k,\ f)} = \{(x,y,z) \in \mathbb{R^3} : f(x,y,z) = k\}
$$
and we're interested in the points
$$ 
V_{(k,\ f)}(\mathbb{Z}) = \{(x,y,z) \in \mathbb{Z^3} : f(x,y,z) = k\}.
$$


Some examples of cubic polynomials which characterise cubic surfaces and explicit cubic surfaces that are of interest in this area are: the sum of three cubes polynomial $F(x,y,z) = x^3 + y^3 + z^3$, the Markov Surfaces $M(x,y,z) = x^2 + y^2 + z^2 - xyz$ and various others such as $(ax+1)(bx+1) + (cy+1)(dy+1) = xyz +1$ where $a, b, c, d$ integers and $a(x^3 + y^3) + z^3 = 1$ where $a$ is also an integer. 

Integer solutions to $x^3 + y^3 + z^3 = k$ ( for $k \not\equiv 4,5\ (\textrm{mod}\ 9)$ ) have even generated popular interest, with various videos by [Numberphile](https://youtube.com/playlist?list=PLdaxdtlgJnT8DLagF9psAYVJZ2bs9ggGB&si=O4alN2Zvytfhx725) on this topic amassing thousands of views. The videos highlight that cubic surfaces are very much an active area of research and very little is known in general: even for these fairly simple equations only as recently as 2019 was an integral solution found for $k = 42$ involving $17+$ digit numbers, the last $k < 100$ such that $k \not\equiv 4,5\ (\textrm{mod}\ 9)$ for an integral solution to be found.

It has been conjectured By Roger Heath-Brown, as highlighted in [this](https://www.pnas.org/doi/10.1073/pnas.2103697118) \[1\] article  that for fixed $k \not\equiv 4,5\ (\textrm{mod}\ 9)$ writing $N(B)$ as the number of integral solutions within the box $max(|x|,|y|,|z|) \leq B$, then we have the asymptotic relation, 
$$ N(B) \sim \delta_{k} \log{B} $$
where $\delta_k$ is a constant which depends only on $k$. If we were to assume this conjecture to be true, then it's no suprise that in this example such computational effort had to be taken to find the first integer solutions for particular values of k. 

Very little is known about cubic surfaces in general so the study of particular examples of cubic surfaces is a worthwhile task. My main focus has been to develop methods to find integral points and look for parametric solutions that are of the form $c(t) = ( x(t), y(t), z(t) )$ where  $x, y, z$ are polynomials with integer coefficients, on the particular cubic surface $x^3 + y^3 + z^3 = 1$, the reason being is that this surface has a lot of integral points to work with. This leads us on to providing an algorithm to find such integer solutions within a given range.   

## 2.  $\boldsymbol{x^3 + y^3 + z^3 = 1}$ 
### 2.1 Algorithms for finding integral points 
### 2.1.1 Brute Force

**Remark.** Before we proceed it's worth pointing out the trivial set of solutions $(x, -x, 1)$ which we rule out any solution set. Also, by the symmetry of the equation if $(x,y,z)$ is a solution to the equation then so are all permutations of coordinates e.g. $(y,x,z)$, $(y, z, x)$ etc. Treating these solutions as being equivalent we choose as our representative (the form that the solution will be stored in) the solution which is of the form $|x| \geq |y| \geq |z|$. 

As before given a positive integer $B$ we want to find all integral points within the box $max\{|x|,|y|,|z| \} \leq B$ which satisfy the diophantine equation in question. A naive algorithm which works is a brute force algorithm, as given in the code below, which involves  searching through all possible points $(x,y,z)$ with $x, y \ \textrm{and}\ z$ in the range $[-B, B]$ and checking whether the sum of the three cubes is equal to 1 or not. One can easily workout the runtime to be $O(B^3)$.



In [22]:
# Brute force algy
import time
start = time.time()

B = 10^2
c = 0

for x in range(-B, B+1):
    for y in range(-B, B+1):
        for z in range(-B, B+1):
            L = [x, y, z]
            if x^3 + y^3 + z^3 == 1 and not(any([n == 1 for n in L])):
                c += 1
end = time.time()                
print('There are', c, 'non-trivial solutions to the cubic equation for B =', B)
print('This algorithm took', round(end - start, 2), 'seconds to run')

There are 12 non-trivial solutions to the cubic equation for B = 100
This algorithm took 25.52 seconds to run


### 2.1.2 Factorisation algorithm
A preferable algorithm which forms the basis of the algorithms used for the surfaces $x^3 + y^3 + z^3 = k$ involves exploiting the factorisation 
$$x^3 + y^3 = (x+y)(x^2 - xy + y^2).$$
After rearranging we get that our equation < Maybe use tags > is equivalent to $(x+y)(x^2 - xy + y^2) = 1 - z^3$ from which we observe that $x+y$ is a factor of $1 - z^3$. Fixing $z$ and setting $d = |x + y| = |x| + y\ \textrm{sgn}\ x$ we get:
\begin{align*}
\frac{|1 - z^3|}{d} &= x^2 - xy + y^2 = x(2x - (x + y)) + y^2 \\ &= |x|(2|x| - d) + (d- |x|)^2 = 3x^2 - 3d|x| + d^2 .
\end{align*}
So we get a quadratic equation in |x| which we solve at which point we get 
$$ |x| = \frac{1}{2} \left( d \pm \sqrt{\frac{4 |1-z^3| - d^3}{3d}} \right) $$
then using $ d = |x| + y \textrm{sgn}\ x$ we get the same pair of values for $|y|$. Moreover since $ x + y = \textrm{sgn}\ (1 - z^3) \textrm{d}$ we then deduce that 
$$ \{ x, y \} = \left\{ \frac{1}{2} \textrm{sgn}\ (1-z^3) \left( d \pm \sqrt{\frac{4 |1-z^3| - d^3}{3d}} \right) \right\}.$$

Hence, for a fixed $z$, we have a way of finding all the corresponding values of $x$ and $y$ by iterating through all the divisiors of $|1-z^3|$. With this algorithm we then construct a function which outputs all solutions for a particular choice of B, ordered by increasing absolute value of z. This algorithm has a much better runtime of $O(B^{1 + \epsilon})$ (although dependent on the time complexity of integer factorisation). 

In [6]:
## Factorisation Algorithm

import time
start = time.time()

def solutions(B):
    S = []
    c = 0
    for z in range(-B, B+1):
        if z in [0, 1]:
            continue
            
        n = ZZ(abs(1-z^3))
        sgn = sign(1-z^3)

        
        S_z = [[d, sqrt(12*(n // d) - 3*d^2)] for d in n.divisors() if                 
is_square(12*(n // d) - 3*d^2)] 
        
        if S_z == []:
            continue
            
        for r in S_z:
            x = sgn*((3*r[0] + r[1]) // 6); y = sgn*((3*r[0] - r[1]) // 6)
            assert x^3 + y^3 + z^3 == 1
            if abs(x) <= B and abs(z) <= min(abs(x), abs(y)):
                S = S + [[x, y, z]]
                c += 1
    
    S = sorted(S, key = lambda c: [abs(c[2]), abs(c[1])])
    
    return S

B = 10^5
a = solutions(B)
end = time.time()

print('the algorithm took', round(end - start), 'seconds to find all non-trivial solutions for B =', B )
print('there are', len(solutions(B)), 'non-trivial solutions for B =', B)
print(solutions(B))

the algorithm took 45.0 seconds to find all solutions for B = 100000
there are 77 non-trivial solutions for B = 100000
[[1, 1, -1], [9, -8, -6], [-12, 10, 9], [-103, 94, 64], [144, -138, -71], [-150, 144, 73], [172, -138, -135], [-249, 235, 135], [1210, -1207, -236], [729, -720, -242], [-738, 729, 244], [-495, 438, 334], [-1544, 1537, 368], [505, -426, -372], [577, -486, -426], [904, -823, -566], [2304, -2292, -575], [-2316, 2304, 577], [1010, -812, -791], [-1988, 1897, 1010], [-1852, 1738, 1033], [5625, -5610, -1124], [-5640, 5625, 1126], [8703, -8675, -1851], [3097, -2820, -1938], [6756, -6702, -1943], [11664, -11646, -1943], [-11682, 11664, 1945], [6081, -5984, -2196], [3753, -3230, -2676], [21609, -21588, -3086], [-21630, 21609, 3088], [-4184, 3518, 3097], [16849, -16806, -3318], [24987, -24965, -3453], [-5262, 4528, 3753], [-9953, 9735, 3987], [-8657, 8343, 4083], [36864, -36840, -4607], [-36888, 36864, 4609], [-38823, 38782, 5700], [-9791, 9036, 5856], [59049, -59022, -6560], [-5

For further optimisations to this algorithm see the paper by Andrew Brooker [here](https://arxiv.org/abs/1903.04284) \[2\].

### 2.2 Parametrisations

Now that we have generated a number of points, it's natural to ask if any of these points are related to one another e.g. if there exists a curve on our surface which they all lie on. One type of curves that we will be interested in are curves of the form $c(t) = (x(t), y(t), z(t))$ where $x(t), y(t)\ \textrm{and}\ z(t)$ are all polynomials with integer coefficients. If such a curve, which we'll call a polynomial parametrisation, exists, then we can generate infinitely many integral points that lie on our surface corresponding to different integer values of $t$.    

**Remark.** There is not much we can immediately deduce about our polynomials knowing that they lie on the surface $x^3 + y^3 + z^3 = 1$, but something we can deduce is that, as a consequence of [Fermat's Last Theorem](https://en.wikipedia.org/wiki/Fermat%27s_Last_Theorem) for the case $n = 3$, we must have that two of the polynomials are of the same degree $d$, say $x(t)\ \textrm{and}\ y(t)$, the other polynomial $z(t)$ is of degree $ < d$ and for the coefficients $a_d \ \textrm{and}\ b_d$ of the leading terms of $x(t)\ \textrm{and}\ y(t)$ respectively then we have $a_d = -b_d$. 

In 2 dimensions, one way of determining a polynomial that runs through a number of points is to use Lagrange Interpolation. Generally speaking, given $n+1$ points $ \{(t_i, x_i)\}^{n+1}_{i=1}$, there exists a unique polynomial $L(x)$ of lowest degree $d \leq n$ which goes through all the $n+1$ points i.e. $L(t_i) = x_i$ for all $1 \leq i \leq n+1$. For information about computing Lagrange Polynomials using sage, see [here](https://doc.sagemath.org/html/en/reference/polynomial_rings/sage/rings/polynomial/polynomial_ring.html#sage.rings.polynomial.polynomial_ring.PolynomialRing_field.lagrange_polynomial).

So say we have picked $n+1$ integral points $ \{(x_i, y_i, z_i)\}^{n+1}_{i=1}$ that we have found on our surface, and we associate each point $p_i = (x_i, y_i, z_i)$ with a t-value $t_i$, then we can produce the coordinate Lagrange polynomials $L_x(t), L_y(t), L_z(t)$ with degrees $ \leq n$, as in the previous paragraph, giving us the curve $c(t) = (L_x(t), L_y(t), L_z(t))$. We then have to check two things for $c(t)$ to be a valid polynomial parametrisation: that $c(t)$ is actually a curve in our surface - $(L_x(t))^3 + (L_y(t))^3 + (L_z(t))^3 = 1$, and that the polynomials have only integer coefficients. 





### 2.2.1 Continued Fractions

For now, lets assume that polynomial parametrisations actually exist and that we don't know what they are. With a given set of integral points on the surface we want to be able to deciper some of these parametrisations with this data. Given a particular parametrisation $c(t) = (x(t), y(t), z(t))$ we'd firstly like to know (i) the degrees of the polynomials $x(t), y(t), z(t)$ and secondly,(ii), we'd like to be able to deduce which of the points actually lie on a given parametrisation; (i) deciding how many points to pick and (ii) what points to pick for the Lagrange Polynomial method. It is then also a question of (iii) what t-values to associate with our chosen points.

To tackle (i), this will involve mainly analysing properties of points in isolation e.g. comparing coordinates with each other. Say we have $c(t) = (x(t), y(t), z(t))$ with $x(t)$ and $y(t)$ of degree $d$ and $z(t)$ of degree $e \leq d-1$, $x(t) = a_d t^d + a_{d-1} t^{d-1} ... + a_0$, $z(t) = b_e t^e + b_{e-1}t^{e-1} + ... + b_0$, then for large $t$ and small $a_d$ and $b_e$ we have the very crude approximation, 
\begin{align*}
 \frac{\log(|x(t)|)}{\log(|z(t)|)} \approx \frac{\log(|a_d t^d|)}{\log(|b_e t^e|)} &= \frac{\log(|a_d|) + d\log(|t|)}{\log(|b_e|) + e\log(|t|)} \\ &\approx \frac{d}{e}.
\end{align*}

We can then use the elegant method of continued fractions to try and capture $\frac{d}{e}$. We can think of continued fractions as an alternative way to representive real numbers instead of via decimal expansion, it has the same 'essentially unique' property and even better than decimals, rational numbers have a finite representation. Given a real number $a$, the continued fraction of $a$ is given by a sequence of real numbers denoted by $[a_0; a_1, a_2, ...]$ which expresses the identity:

$$ a = [a_0; a_1, a_2, ...] = a_0 + \frac{1}{a_1 + {\frac{1}{a_2 + \frac{1}{a_3 + \frac{1}{...}}}}}.
$$
For a rational number $\frac{p}{q}$ we have an 'essentially unique' **finite** representation, in that every rational number can be expressed in only two ways,  $[a_0; a_1, ..., a_{n-1}, a_n] = [a_0;a_1, ..., a_{n-1}, (a_n - 1), 1]$ with the first representation typically chosen, 

$$ \frac{p}{q} = [a_0; a_1, ..., a_{n-1}, a_n] = a_0 + \frac{1}{a_1 + \frac{1}{a_2 + \frac{1}{\ddots + \frac{1}{a_n}}}}.$$

In contrast, irrational numbers have a fully unique infinite representation. The convergents $c_k$ for $0 \leq k \leq n$ of a continued fraction $ \frac{p}{q} = [a_0; a_1, ..., a_{n-1}, a_n]$ are the truncated terms $ c_0 = a_0$ and $ c_k = [a_0; a_1, ..., a_k]$ for $1 \leq k \leq n$ which provide increasingly better approximations to our fraction.  

Below is an example computing the continued fraction of the rational number $\frac{23}{157}$ in sage.




In [2]:
frac = 337/153
a = continued_fraction(frac)
                       
print('The continued fraction representation of', frac, 'is:',a)
                       
print('The convergents are:',a.convergents())

The continued fraction representation of 337/153 is: [2; 4, 1, 14, 2]
The convergents are: [2, 9/4, 11/5, 163/74, 337/153]


So given a point $(x, y, z)$ in our set of solutions, we compute $\frac{\log(x)}{\log(z)}$ which returns a floating point value which we can convert into a rational $\frac{a}{b}$. For $\frac{a}{b}$ we can then compute its finite continued fraction along with its convergents. 

As the parametrisations that we're interested in will have relatively small degree (too large and there won't be enough points within our data set to apply our Lagrange Interpolation method on), if we have that a fraction $\frac{p}{q}$ with small $p$ and $q$ occurs often in the set of convergents associated with a particular point or that sometimes $\frac{p}{q}$ provides a very good approximation to $\frac{\log(x)}{\log(z)}$, then at the very least we have suspicion there exists a parametrisation where $x(t)$ and $y(t)$ have degree of the form $ld$ and $z(t)$ of degree $le$ with $l > 0$. We'd be able to pick out quite easily if a particular term of the convergents $c_k$  provides a very good approximation to $\frac{\log(x)}{\log(z)}$ if the next term $c_{k+1}$ has a much larger numerator and denominator, as a result of a very large coefficient $a_{k+1}$ . As the example below highlights, we can very easily pick out that $\frac{3}{2}$ provides a very good approximation to the value $1.500217$, as we'd expect.

In [7]:
a = continued_fraction(1.500217)
print(a)
print(a.convergents())

[1; 1, 1, 1151, 1, 1, 2, 1, 8, 7]
[1, 2, 3/2, 3455/2303, 3458/2305, 6913/4608, 17284/11521, 24197/16129, 210860/140553, 1500217/1000000]


Now we will put this idea into practice. Many thanks to Tim Browning for providing Andrew Sutherland's collection of solutions to the equation for $B = 10^{15}$ containing around $10^5$ points for us to work with. We'll pick 100 of these points at random to apply this method on to see if there's anything to observe. 

In [13]:
#Andrew Sutherland's solutions
import pickle

with open('solutions','rb') as fp:
    sols = pickle.load(fp)   #solutions for B = 10^15
    

99678


In [21]:
import random
import pickle

%%capture cap


# Data set for min(|x|, |y|, |z|) < 10^15
with open('solutions','rb') as fp:
    sols = pickle.load(fp) 

# Continued fraction for 5 random points of our data set      
for i in random.choices(sols, k = 5):
    x = i[0]; z = i[2]
    r = RR(log(abs(x/9))/log(abs(z/9)))
    cont_frac = QQ(r).continued_fraction() 
    convs = cont_frac.convergents()
    print('(x, y, z) =', i)
    print('continued fraction:', cont_frac)
    print('convergents:',convs)
    print()

with open('output.txt', w) as file:
    file.write(cap.stdout)
    
    
    

UsageError: Line magic function `%%capture` not found.


One observation that is pretty immediate is the frequent occurence of $\frac{4}{3}$ as the second convergent of the continued fractions. This suggests that we at least try to see if there is such a parametrisation $(x(t), y(t), z(t))$ where $x(t)$ and $y(t)$ of degree 4 and $z(t)$ of degree 3 (which we'll call a quartic parametrisation from now on) and if not then degrees  $(8,8,6)$, $(12, 12, 9)$ and so on. Now we grapple with the task (iii) as mentioned at the start of this section: what t-values to associate with points that we assume to be on our curve.

### 2.2.2 Pairs of points



For the continued fraction method we looked at points in isolation, now to find parametrisations, we ought to want to consider pairs of points. More precisely we want to find consecutive points and here's why:

Say we have a parametrisation $c(t) = (x(t), y(t), z(t))$, where $x(t)$ and $y(t)$ are of degree n and $z(t)$ of degree $m \leq n$, and two consecutive integral points $(x_1, y_1, z_1) = (x(t_0), y(t_0), z(t_0))$, $(x_2, y_2, z_2) = (x(t_0+1), y(t_0+1), z(t_0+1))$, with $t_0$ a positive integer, on the curve. Considering the $x$-coordinate, we have for large $t_0$ the approximation,
\begin{align*}
\left|\frac{x_2}{x_1}\right| = \left|\frac{a_n(t_0+1)^n + a_{n-1}(t_0+1)^{n-1} + ... + a_0}{a_n(t_0)^n + a_{n-1}t^{n-1} ... + a_0}\right| &\approx \left| \frac{a_n(t_0 + 1)^n}{a_nt_0^n} \right| \\ &= {\left | 1 + \frac{1}{t_0} \right |}^n 
\end{align*}

Taking the nth-root of the right hand side we get $1 + \frac{1}{t_0}$, then after taking the fractional part of this expression and then reciprocating we get back $t_0$. This also works for $y(t)$ and $z(t)$ provided we instead take the m-th root for $z(t)$.


For consecutive negative t-values i.e. $-t_0$, $-(t_0 + 1)$ for $t_0 > 0$, then the above still works, we just get back $t_0$ instead. As we shall see, ultimately this shouldn't be an issue, because if $c(t)$ is a polynomial parametrisation then $c(-t)$ is also.

We now present a summary of a program that puts this idea into practice to see if we can find a quartic parametrisation as the data is hinting at.   


#### Summary of the program:
**- Finding candidate consecutive points:**
- Start out with a particular point $p_1$ and we want to find a sensible candidate for its consecutive point $p_2$. 
- For each point, since |x| >= |y| >= |z|, point $p_1$ is of a particular form - either (+, -, -) or (-, +, +) where the symbols represent the signs of the coordinates. 
- Without loss of generality, say $p_1$ is of the form (+, -, -), then since our list is ordered such that |z| is increasing, we pick the next p2 in our list of solutions which is of the same form (+, -, -) as our candidate consecutive point. 
- We then use the ideas above to compute candidate t-values $t_x, t_y, t_z$ for each of the coordinates. 
- That is, assuming that the points lie on a quartic parametrisation, we compute the ratio of the two points, take the   4th root for x and y coordinate and 3rd root for z coordinate, take the fractional part of this, reciprocate and       choosing the nearest integer to these values to obtain $t_x, t_y\ \textrm{and}\ t_z$.
- We expect the t-values that we compute to be at-least pretty similar if $p_1$ and $p_2$ do indeed lie on a quartic     parametrisation e.g. that $t_z$ is equal to $t_x$ or $t_y$. 

**- Once we have such a pair:**
- If so then these points are stored in a list in a dictionary consisting of all such lists.
- We have a dictionary which contains lists of 'consecutive' points.
- If there is a list with $p_1$ at the end it, we add $p_2$ along with its corresponding t-value $t_z +1$ to the end of such a list.
- If not then we start a new list containing $p_1$ and $p_2$ along with their t-values $t_z$ and $t_z + 1$.
- Once a list has 5 points we then compute the lagrange polynomials $L_x(t),\ L_y(t),\ L_z(t)$ from these 'consecutive' points along with their stored t-values.
- We then check if $c(t) = (L_x(t), L_y(t), L_z(t)$ is indeed a quartic parametrisation or not.



**The program:**

In [31]:
# Algorithm for finding quartic parametrisation 
import time
import sys

start = time.time()

my_dict = {} #Dicitonary to store lists of consecutive points
L = len(sols) #sols = list of solutions for B = 10^15
R = PolynomialRing(QQ,'t')

for i in range(L):
    c1 = i; c2 = i+1
    x1, y1, z1 = sols[c1]; x2, y2, z2 = sols[c2] #consecutive points
    r_z = abs(z2/z1)^(1/3)
    
    while r_z <= 1.2: #ensures that t >= 5
        x1, y1, z1 = sols[c1]; x2, y2, z2 = sols[c2]
        r_z = RR((abs(z2/z1)^(1/3)))
        if abs(x1) >= abs(x2) or sign(x1) != sign(x2):
            c2+=1; continue
            
        r_x = RR((abs(x2/x1)^(1/4))); r_y = RR((abs(y2/y1)^(1/4)));r_z = RR((abs(z2/z1)^(1/3)))
        if r_x in ZZ or r_y in ZZ or r_z in ZZ:
            c2+=1; continue
            
        frac_x = QQ(frac(r_x)); frac_y = QQ(frac(r_y)); frac_z = QQ(frac(r_z))
        t_x = (1/frac_x).round(); t_y = (1/frac_y).round(); t_z = (1/frac_z).round()
        
        #Process of adding point(s) to list in dictionary:
        
        if t_z == t_x or t_z == t_y:
               
            list1 = [] #pairs of start and end points of lists of consec pts
            for i in my_dict.keys():
                list1 += [[i, my_dict[i][-1]]]
            list2 = [x[1] for x in list1] #list of all end points
            
            if not [t_z, sols[c1]] in list2: 
                my_dict[(t_z, tuple(sols[c1]))] = [[t_z, sols[c1]], [t_z + 1, sols[c2]]]
            
            elif [t_z, sols[c1]] in list2:
                index = list2.index([t_z, sols[c1]])
                key = list1[index][0]
                my_dict[key] += [[t_z + 1, sols[c2]]]
                
                #Computing Lagrange polys with 5 consecutive points:
                
                if len(my_dict[key]) == 5: 
                    L_x(t) = R.lagrange_polynomial([(n[0],n[1][0]) for n in my_dict[key]])
                    L_y(t) = R.lagrange_polynomial([(n[0],n[1][1]) for n in my_dict[key]])
                    L_z(t) = R.lagrange_polynomial([(n[0],n[1][2]) for n in my_dict[key]])
                    c(t) = (L_x(t), L_y(t), L_z(t))
                    
                    c_coeffs = L_x.coefficients() + L_y.coefficients() + L_z.coefficients()
                    if not all(x[0] in ZZ for x in c_coeffs):
                        my_dict.pop(key); c2+=1; continue
                        
                    three_cubes = (L_x(t))^3 + (L_y(t))^3 + (L_z(t))^3
                    if three_cubes != 1:
                        my_dict.pop(key); c2+=1; continue
                    elif three_cubes == 1:
                        end = time.time()
                        print('- We have found the quartic parametrisation:', c(t))
                        print('- It took', round(end - start,2), 'seconds to find such a parametrisation')
                        print('- The consecutive points and t-values used to find this parametrisation are:' \
                              , my_dict[key])
                        
                        pts = [x[1] for x in my_dict[key]]
                        indexes = []
                        for i in pts:
                            a = sols.index(i)
                            indexes += [a]
                        print('- These points have corresponding entries in our list:', indexes)
                        sys.exit('found quartic parametrisation')
                        
        c2+=1
                    
    #Remove invalid entries from the dictionary:
    
    for i in my_dict.copy():
        if my_dict[i][-1][1] == sols[c1]:
            my_dict.pop(i, None)


    






- We have found the quartic parametrisation: (-9*t^4 - 3*t, 9*t^4, 9*t^3 + 1)
- It took 0.17 seconds to find such a parametrisation
- The consecutive points and t-values used to find this parametrisation are: [[5, [-5640, 5625, 1126]], [6, [-11682, 11664, 1945]], [7, [-21630, 21609, 3088]], [8, [-36888, 36864, 4609]], [9, [-59076, 59049, 6562]]]
- These points have corresponding entries in our list: [22, 27, 31, 39, 43]


SystemExit: found quartic parametrisation

So we get the same parametric solution (after rearrangement and change of parametrisation) as the first one, $(9t^4, 3t - 9t^4, 9t^3 + 1)$, that is presented in a [paper](https://londmathsoc.onlinelibrary.wiley.com/doi/abs/10.1112/jlms/s1-31.3.275) \[3\] by D.H. Lehlmer. This paper actually gives an account of an *infinite* number of polynomial parametrisations, these parametrisations generated by a system of recurrence relations. 

The question now is can we find any of the other ones. Before we move on to answering this, lets have a look at the continued fraction method but this time with the coefficients of the leading terms considered of $x(t)$ and $z(t)$, in this case both $9$. In other words, instead of considering $\frac{\log(|x|)}{\log(|z|)}$, we now consider $\frac{\log(|\frac{x}{9}|)}{\log(|\frac{z}{9}|)}$ which should provide a much better approximation to $\frac{4}{3}$ if our point $(x, y, z)$ does indeed lie on this parametrisation, as our previous more crude approximation simply ignored these coefficients. 

In [6]:
import random

for i in random.choices(sols, k = 5):
    x = i[0]; z = i[2]
    r = RR(log(abs(x/9))/log(abs(z/9)))
    cont_frac = QQ(r).continued_fraction()
    convs = cont_frac.convergents()
    print('(x, y, z) =', i)
    print('continued fraction:', cont_frac)
    print('convergents:',convs)
    print()
print('finished')

(x, y, z) = [-20266878519300, 20266878515625, 16544390626]
continued fraction: [1; 2, 1, 23528119110]
convergents: [1, 3/2, 4/3, 94112476443/70584357332]

(x, y, z) = [5118089805132510969, -5118089805132428586, -186376672558628]
continued fraction: [1; 2, 1, 428914250225761]
convergents: [1, 3/2, 4/3, 1715657000903047/1286742750677285]

(x, y, z) = [6228797746985866809, -6228797746985780280, -215955266337962]
continued fraction: [1; 2, 1, 428914250225761]
convergents: [1, 3/2, 4/3, 1715657000903047/1286742750677285]

(x, y, z) = [10554762465547409664, -10554762465547310940, -320735458415807]
continued fraction: [1; 2, 1, 428914250225761]
convergents: [1, 3/2, 4/3, 1715657000903047/1286742750677285]

(x, y, z) = [4987405419572001024, -4987405419571919172, -182795976380735]
continued fraction: [1; 2, 1, 230953827044641]
convergents: [1, 3/2, 4/3, 923815308178567/692861481133925]

finished


The above shows that this approximation is indeed much better for points on the parametrisation $(9t^4, 3t - 9t^4, 9t^3 + 1)$ than before and we can say with pretty much full confidence which of the points lie on this parametrisation. Although the task of actually finding the leading coefficients, given that we know a parametrisation with certain powers exists, seems a much harder task.

### 2.2.3 Non-quartic points

Before we go on to see if this method works to find any other parametrisations presented by Lehmer, lets see what we can say about the points that don't lie on this quartic parametrisation. 

In [26]:
import pickle
with open('other_solutions', 'rb') as fp:
    other_sols = pickle.load(fp) #Solutions not on the quartic parametrisation
  

print('- The number of non-quartic solutions is:', len(other_sols))
print('- Compared with the total number of solutions:', len(sols))

- The number of non-quartic solutions is: 3530
- Compared with the total number of solutions: 99678


As we see, it seems that most of the points lie on this quartic parametrisation. It should therefore be a less straightforward task of finding the other parametrisations. We can again try the primitive continued fraction method on the non-quartic method to see if we can find anything that we don't already know. 

It's worth investigating the second convergents in the continued fraction as this proved useful for deducing that there was a quartic parametrisation. Below I construct an algorithm that counts the number of times fractions occur as the second convergents. We'll restrict to fractions $\frac{p}{q}$ that have numerator $p < 17$, any more then we won't have enough points to produce such a degree-n parametrisation, $n \geq 16$, via Langrange Interpolation anyway.

In [27]:
second_convs = []
L = len(other_sols)
for i in range(L):
    x = other_sols[i][0]; z = other_sols[i][2]
    r = RR(log(abs(x))/log((abs(z))))
    cont_frac = QQ(r).continued_fraction()
    convs = cont_frac.convergents()
    
    if convs[1].numerator() >= 17:
        continue
    
    
    fracs1 = [x[0] for x in second_convs]
    if convs[1] in fracs1:
        index1 = fracs1.index(convs[1])
        second_convs[index1][1] += 1
    else:
        second_convs += [[convs[1], 1]]

second_convs = sorted(second_convs, key = lambda k: [k[0].numerator(), k[1]])

print('second convergents and times they occur ([p/q, n]):',second_convs)

second convergents and times they occur ([p/q, n]): [[3/2, 2], [4/3, 129], [5/4, 24], [6/5, 23], [7/6, 52], [8/7, 53], [9/8, 65], [10/9, 66], [11/10, 75], [12/11, 83], [13/12, 102], [14/13, 90], [15/14, 85], [16/15, 78]]


There's not much to immediately pick out from this data, except the fact that it now seems very unlikely that there's either quintic or degree-6 parametrisations, the quantities for the other fractions don't really seem to be telling us that much, there's no particular fraction that seems to stand out; it doesn't hint at there being a degree-10 or degree-16 parametrisation. 

Nevertheless lets see if our method is able to find the degree-10 parametrisation. The algorithm will be virtually the same as the one which found the quartic parametrisation:


In [32]:
# Algorithm for finding deg-10 parametrisation 
import time
import sys
start = time.time()

sols = other_sols
my_dict = {} #Dicitonary to store lists of consecutive points
L = len(sols) #sols = list of solutions for B = 10^15
R = PolynomialRing(QQ,'t')

for i in range(round(L/4),L-1):
    c1 = i; c2 = i+1
    x1, y1, z1 = sols[c1]; x2, y2, z2 = sols[c2] #consecutive points
    r_z = abs(z2/z1)^(1/9)
    
    
    while r_z <= 1.2 and c2 <= L-1: #ensures that t >= 5
        x1, y1, z1 = sols[c1]; x2, y2, z2 = sols[c2]
        r_z = RR((abs(z2/z1)^(1/9)))
        if abs(x1) >= abs(x2) or sign(x1) != sign(x2):
            c2+=1; continue
            
        r_x = RR((abs(x2/x1)^(1/10))); r_y = RR((abs(y2/y1)^(1/10)));r_z = RR((abs(z2/z1)^(1/9)))
        if r_x in ZZ or r_y in ZZ or r_z in ZZ:
            c2+=1; continue
            
        frac_x = QQ(frac(r_x)); frac_y = QQ(frac(r_y)); frac_z = QQ(frac(r_z))
        t_x = (1/frac_x).round(); t_y = (1/frac_y).round(); t_z = (1/frac_z).round()
        
        #Process of adding point(s) to list in dictionary:
        
        if t_z == t_x or t_z == t_y:
               
            list1 = [] #pairs of start and end points of lists of consec pts
            for i in my_dict.keys():
                list1 += [[i, my_dict[i][-1]]]
            list2 = [x[1] for x in list1] #list of all end points
            
            if not [t_z, sols[c1]] in list2: 
                my_dict[(t_z, tuple(sols[c1]))] = [[t_z, sols[c1]], [t_z + 1, sols[c2]]]
            
            elif [t_z, sols[c1]] in list2:
                index = list2.index([t_z, sols[c1]])
                key = list1[index][0]
                my_dict[key] += [[t_z + 1, sols[c2]]]
                
                #Computing Lagrange polys with 11 consecutive points:
                
                if len(my_dict[key]) == 11: 
                    L_x(t) = R.lagrange_polynomial([(n[0],n[1][0]) for n in my_dict[key]])
                    L_y(t) = R.lagrange_polynomial([(n[0],n[1][1]) for n in my_dict[key]])
                    L_z(t) = R.lagrange_polynomial([(n[0],n[1][2]) for n in my_dict[key]])
                    c(t) = (L_x(t), L_y(t), L_z(t))
                    
                    c_coeffs = L_x.coefficients() + L_y.coefficients() + L_z.coefficients()
                    if not all(x[0] in ZZ for x in c_coeffs):
                        my_dict.pop(key); c2+=1; continue
                        
                    three_cubes = (L_x(t))^3 + (L_y(t))^3 + (L_z(t))^3
                    if three_cubes != 1:
                        my_dict.pop(key); c2+=1; continue
                    elif three_cubes == 1:
                        end = time.time()
                        print('- We have found the quartic parametrisation:', c(t))
                        print('- It took', round(end - start,2), 'seconds to find such a parametrisation')
                        print('- The consecutive points and t-values used to find this parametrisation are:', \
                              my_dict[key])
                        
                        pts = [x[1] for x in my_dict[key]]
                        indexes = []
                        for i in pts:
                            a = sols.index(i)
                            indexes += [a]
                        print('- These points have corresponding entries in our list:', indexes)
                        sys.exit('found degree 10 parametrisation')
                        
        c2+=1
                    
    #Remove invalid entries from the dictionary:
    
    for i in my_dict.copy():
        if my_dict[i][-1][1] == sols[c1]:
            my_dict.pop(i, None)


    






/tmp/ipykernel_3860184/1154042592.py:11: DeprecationWarning: the default rounding for rationals, currently `away`, will be changed to `even`.
See https://github.com/sagemath/sage/issues/35473 for details.
  for i in range(round(L/Integer(4)),L-Integer(1)):


- We have found the quartic parametrisation: (3888*t^10 - 135*t^4, -3888*t^10 + 1296*t^7 - 81*t^4 - 3*t, -3888*t^9 + 648*t^6 + 9*t^3 + 1)
- It took 435.08 seconds to find such a parametrisation
- The consecutive points and t-values used to find this parametrisation are: [[7, [1098263443977, -1097196650886, -156818584376]], [8, [4174707658752, -4171990634520, -521668652543]], [9, [13556616865353, -13550419554732, -1505946480902]], [10, [38879998650000, -38867040810030, -3887351990999]], [11, [100844704872153, -100819452661026, -9166552639100]], [12, [240734709303552, -240688275759396, -20059291075391]], [13, [535993812453177, -535912496544360, -41227165770218]], [14, [1124622093360528, -1124485485757242, -80325270732167]], [15, [2242016711915625, -2241795289100670, -149460400094624]], [16, [4274901199945728, -4274553321750576, -267170453876735]], [17, [7838184273670377, -7837652492790756, -461054022631406]]]
- These points have corresponding entries in our list: [1186, 1404, 1628, 1824,

SystemExit: found degree 10 parametrisation

As we see, this program was able to find the degree-10 parametrisation, 
$$  (3888t^{10} - 135t^4, -3888t^{10} + 1296t^7 - 81t^4 - 3t, -3888t^9 + 648t^6 + 9t^3 + 1) $$
that is equivalent to the one given as the second term of the recurrence relation in Lehmer's [paper](https://londmathsoc.onlinelibrary.wiley.com/doi/epdf/10.1112/jlms/s1-31.3.275). This appears to be the limit of what this algorithm can do with the given solutions, for the parametrisation with degree-16 or higher  at most 10 points of either form $(+,-,-)$ or $(-, +, +)$ will be present in the solutions I've been provided with, not enough to perform Lagrange Interpolation on. 



## 3. Conclusion 

This project was defintely not intended to go down this route, it was never the plan to spend the majority of time exploring the cubic surface $x^3 + y^3 + z^3 = 1$, although in doing so I have learnt a lot and enjoyed being able to freely experiment with different ideas that I had only encountered before in a purely theoretic setting. 

Although a large part of the challenge is actually finding solutions on cubic equations in the first place, it would certainly be interesting for me to see if these ideas could be used on other cubic surfaces which contain lots of solutions. 





## 4. Bibliography

\[1\] Siksek, Samir. "Sums of integer cubes." Proceedings of the National Academy of Sciences 118.16 (2021):                 e2103697118.

\[2\] Booker, Andrew R. "Cracking the problem with 33." Research in Number Theory 5 (2019): 1-6.

\[3\] Lehmer, D. H. "On the Diophantine Equation x3+ y3+ z3= 1." Journal of the London Mathematical Society 1.3             (1956): 275-280.